# Moderator Mining

We are creating our home brew index for moderating statements.
- [ ] compute the variance of author timelines for a set of tweets (topic_var)
- [ ] compute the sentiment delta function for each tweet
- [ ] compute the topic delta function  for each tweet
- [ ] compute unweighted moderator index
- [ ] display the tweets with the highest m_index rating
